
Dengan menggunakan kumpulan data harga mobil bekas yang dilampirkan, lakukan hal berikut:

*Muat data “used_car_price.csv”

*Pisahkan data menjadi 75% untuk pelatihan dan 25% untuk pengujian

*Latih model XG-Boost di Scikit-Learn

*Nilai kinerja model XG-Boost terlatih menggunakan RMSE dan R2

*Lakukan optimasi hiperparameter menggunakan GridSearch, pilih nilai yang masuk akal untuk max_depth, learning_rate, n_estimators, dan colsample_bytree. Gunakan 5-folds cross validation.

*Lakukan optimasi hiperparameter menggunakan RandomSearch, pilih nilai yang masuk akal untuk max_depth, learning_rate, n_estimators, dan colsample_bytree. Gunakan 5-folds cross validation dan 100 iterasi.

*Lakukan optimasi hiperparameter menggunakan Bayesian optimization, pilih nilai yang masuk akal untuk max_depth, learning_rate, n_estimators. Gunakan 5-folds cross validation dan 100 iterasi.

*Bandingkan 3 strategi pengoptimalan menggunakan RMSE dan R2.



In [1]:
#Muat data “used_car_price.csv”
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
car_df = pd.read_csv('used_car_price.csv')
car_df.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945,3.5,6,265,17,23,4451,106,189
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820,2.0,4,200,24,31,2778,101,172
2,Acura,TSX 4dr,Sedan,Asia,Front,26990,2.4,4,200,22,29,3230,105,183
3,Acura,TL 4dr,Sedan,Asia,Front,33195,3.2,6,270,20,28,3575,108,186
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755,3.5,6,225,18,24,3880,115,197


In [3]:
car_df.tail()

,Make,Model,Type,Origin,DriveTrain,MSRP,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
423,Volvo,C70 LPT convertible 2dr,Sedan,Europe,Front,40565,2.4,5,197,21,28,3450,105,186
424,Volvo,C70 HPT convertible 2dr,Sedan,Europe,Front,42565,2.3,5,242,20,26,3450,105,186
425,Volvo,S80 T6 4dr,Sedan,Europe,Front,45210,2.9,6,268,19,26,3653,110,190
426,Volvo,V40,Wagon,Europe,Front,26135,1.9,4,170,22,29,2822,101,180
427,Volvo,XC70,Wagon,Europe,All,35145,2.5,5,208,20,27,3823,109,186


In [4]:
car_df.describe()

,MSRP,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
count,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000
mean,32774.855140,3.196729,5.799065,215.885514,20.060748,26.843458,3577.953271,108.154206,186.362150
std,19431.716674,1.108595,1.559679,71.836032,5.238218,5.741201,758.983215,8.311813,14.357991
min,10280.000000,1.300000,3.000000,73.000000,10.000000,12.000000,1850.000000,89.000000,143.000000
25%,20334.250000,2.375000,4.000000,165.000000,17.000000,24.000000,3104.000000,103.000000,178.000000
50%,27635.000000,3.000000,6.000000,210.000000,19.000000,26.000000,3474.500000,107.000000,187.000000
75%,39205.000000,3.900000,6.000000,255.000000,21.250000,29.000000,3977.750000,112.000000,194.000000
max,192465.000000,8.300000,12.000000,500.000000,60.000000,66.000000,7190.000000,144.000000,238.000000


In [5]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Make         428 non-null    object 
 1   Model        428 non-null    object 
 2   Type         428 non-null    object 
 3   Origin       428 non-null    object 
 4   DriveTrain   428 non-null    object 
 5   MSRP         428 non-null    int64  
 6   EngineSize   428 non-null    float64
 7   Cylinders    428 non-null    int64  
 8   Horsepower   428 non-null    int64  
 9   MPG_City     428 non-null    int64  
 10  MPG_Highway  428 non-null    int64  
 11  Weight       428 non-null    int64  
 12  Wheelbase    428 non-null    int64  
 13  Length       428 non-null    int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 46.9+ KB


In [6]:
car_df.isnull().sum()

Make           0
Model          0
Type           0
Origin         0
DriveTrain     0
MSRP           0
EngineSize     0
Cylinders      0
Horsepower     0
MPG_City       0
MPG_Highway    0
Weight         0
Wheelbase      0
Length         0
dtype: int64

In [7]:
#one hot encoding untuk ['Make', 'Model', 'Type', 'Origin', 'DriveTrain'] var categorical
car_df = pd.get_dummies(car_df, columns = ['Make', 'Model', 'Type', 'Origin', 'DriveTrain'])
car_df

,MSRP,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length,Make_Acura,...,Type_Sedan,Type_Sports,Type_Truck,Type_Wagon,Origin_Asia,Origin_Europe,Origin_USA,DriveTrain_All,DriveTrain_Front,DriveTrain_Rear
0,36945,3.5,6,265,17,23,4451,106,189,1,...,0,0,0,0,1,0,0,1,0,0
1,23820,2.0,4,200,24,31,2778,101,172,1,...,1,0,0,0,1,0,0,0,1,0
2,26990,2.4,4,200,22,29,3230,105,183,1,...,1,0,0,0,1,0,0,0,1,0
3,33195,3.2,6,270,20,28,3575,108,186,1,...,1,0,0,0,1,0,0,0,1,0
4,43755,3.5,6,225,18,24,3880,115,197,1,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,40565,2.4,5,197,21,28,3450,105,186,0,...,1,0,0,0,0,1,0,0,1,0
424,42565,2.3,5,242,20,26,3450,105,186,0,...,1,0,0,0,0,1,0,0,1,0
425,45210,2.9,6,268,19,26,3653,110,190,0,...,1,0,0,0,0,1,0,0,1,0
426,26135,1.9,4,170,22,29,2822,101,180,0,...,0,0,0,1,0,1,0,0,1,0


In [8]:
#masukan fitur input ke X dan output MSRP ke y
X = car_df.drop('MSRP', axis = 1)
y = car_df['MSRP'] #kolom MSRP yang ingin diprediksi

In [9]:
X

,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length,Make_Acura,Make_Audi,...,Type_Sedan,Type_Sports,Type_Truck,Type_Wagon,Origin_Asia,Origin_Europe,Origin_USA,DriveTrain_All,DriveTrain_Front,DriveTrain_Rear
0,3.5,6,265,17,23,4451,106,189,1,0,...,0,0,0,0,1,0,0,1,0,0
1,2.0,4,200,24,31,2778,101,172,1,0,...,1,0,0,0,1,0,0,0,1,0
2,2.4,4,200,22,29,3230,105,183,1,0,...,1,0,0,0,1,0,0,0,1,0
3,3.2,6,270,20,28,3575,108,186,1,0,...,1,0,0,0,1,0,0,0,1,0
4,3.5,6,225,18,24,3880,115,197,1,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,2.4,5,197,21,28,3450,105,186,0,0,...,1,0,0,0,0,1,0,0,1,0
424,2.3,5,242,20,26,3450,105,186,0,0,...,1,0,0,0,0,1,0,0,1,0
425,2.9,6,268,19,26,3653,110,190,0,0,...,1,0,0,0,0,1,0,0,1,0
426,1.9,4,170,22,29,2822,101,180,0,0,...,0,0,0,1,0,1,0,0,1,0


In [10]:
y

0      36945
1      23820
2      26990
3      33195
4      43755
       ...  
423    40565
424    42565
425    45210
426    26135
427    35145
Name: MSRP, Length: 428, dtype: int64

In [11]:
X = np.array(X)

In [12]:
X

array([[  3.5,   6. , 265. , ...,   1. ,   0. ,   0. ],
       [  2. ,   4. , 200. , ...,   0. ,   1. ,   0. ],
       [  2.4,   4. , 200. , ...,   0. ,   1. ,   0. ],
       ...,
       [  2.9,   6. , 268. , ...,   0. ,   1. ,   0. ],
       [  1.9,   4. , 170. , ...,   0. ,   1. ,   0. ],
       [  2.5,   5. , 208. , ...,   1. ,   0. ,   0. ]])

In [13]:
y = np.array(y)

In [14]:
y

array([ 36945,  23820,  26990,  33195,  43755,  46100,  89765,  25940,
        35940,  31840,  33430,  34480,  36640,  39640,  42490,  44240,
        42840,  49690,  69190,  48040,  84600,  35940,  37390,  40590,
        40840,  49090,  37000,  52195,  28495,  30795,  37995,  30245,
        35495,  36995,  37245,  39995,  44295,  44995,  54995,  69195,
        73195,  48195,  56595,  33895,  41045,  32845,  37895,  26545,
        22180,  26470,  24895,  28345,  32245,  35545,  40720,  52795,
        46995,  30835,  45445,  50595,  47955,  76200,  52975,  42735,
        41465,  30295,  20255,  11690,  12585,  14610,  14810,  16385,
        21900,  18995,  20370,  21825,  25000,  27995,  23495,  24225,
        26395,  27020,  44535,  51535,  36100,  18760,  20310,  40340,
        41995,  22225,  17985,  22000,  19090,  21840,  29865,  24130,
        26860,  25955,  25215,  33295,  30950,  27490,  38380,  34495,
        31230,  32235,  13670,  15040,  22035,  18820,  20220,  24885,
      

In [15]:
#Pisahkan data menjadi 75% untuk pelatihan dan 25% untuk pengujian
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

In [16]:
X_train.shape #75% itu 321

(321, 483)

In [17]:
X_test.shape #25% itu 107

(107, 483)

In [18]:
#Latih model XG-Boost di Scikit-Learn
#xgb tanpa optimasi
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import xgboost as xgb
model = xgb.XGBRegressor(objective = 'reg:squarederror', learning_rate = 1, max_depth = 3, n_estimators = 500)
model.fit(X_train, y_train)

result = model.score(X_test, y_test)
y_predict = model.predict(X_test)

In [20]:
#Nilai kinerja model XG-Boost terlatih menggunakan RMSE dan R2
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt

RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict)), '.3f'))
r2 = r2_score(y_test, y_predict)

print('RMSE= ', RMSE, '\nR2= ', r2)

RMSE=  6128.629 
R2=  0.8512921025017814


In [21]:
#Lakukan optimasi hiperparameter menggunakan GridSearch, pilih nilai yang masuk akal untuk max_depth,
#learning_rate, n_estimators, dan colsample_bytree. Gunakan 5-folds cross validation
from sklearn.model_selection import GridSearchCV
#xgb dengan grid search
parameters_grid = { 'learning_rate' : [0.1, 0.5],
                    'max_depth' : [3, 10, 20],
                     'n_estimators' : [100, 500],
                     'colsample_bytree' : [0.3, 0.7]}

model = xgb.XGBRegressor(objective= 'reg:squarederror')

#karena menggunakan neg mse sebab gidsearchcv memeringkat semua algoritma
#dan menentukan mana yg terbaik dengan meminimalkan error
xgb_gridsearch = GridSearchCV(estimator = model,
                              param_grid = parameters_grid,
                              scoring = 'neg_mean_squared_error',
                              cv = 5,
                              verbose = 5)

xgb_gridsearch.fit(X_train, y_train)
y_predict = xgb_gridsearch.predict(X_test)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=-30166248.359 total time=   0.1s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=-216911694.297 total time=   0.1s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=-65477148.638 total time=   0.1s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=-56605213.971 total time=   0.1s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=-28103555.310 total time=   0.1s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=500;, score=-28902669.220 total time=   0.3s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=500;, score=-207494495.197 total time=   0.6s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.1, max_dep

In [22]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt

RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict)), '.3f'))
r2 = r2_score(y_test, y_predict)

print('RMSE =', RMSE, '\nR2 =', r2)

RMSE = 5366.941 
R2 = 0.8859590305351258


In [29]:
#Lakukan optimasi hiperparameter menggunakan RandomSearch, pilih nilai yang masuk akal untuk max_depth,
#learning_rate, n_estimators, dan colsample_bytree. Gunakan 5-folds cross validation dan 100 iterasi
from sklearn.model_selection import RandomizedSearchCV

# Tentukan grid hyperparameter untuk dicari

# Anda dapat memilih booster yang ingin Anda pilih:
# Tersedia dua opsi: gbtree, gblinear
# gbtree menggunakan model berbasis pohon sedangkan gblinear menggunakan fungsi linier

grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 10, 20],
    'learning_rate': [0.1, 0.5],
    'colsample_bytree': [0.3, 0.7]}


import xgboost as xgb
model = xgb.XGBRegressor(objective ='reg:squarederror')

random_cv = RandomizedSearchCV(estimator = model,
                               param_distributions = grid,
                               cv = 5,
                               n_iter = 100,
                               scoring = 'neg_mean_absolute_error',
                               verbose = 5,
                               return_train_score = True)
random_cv.fit(X_train, y_train)

random_cv.best_estimator_
y_predict = random_cv.predict(X_test)


from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt

RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict)),'.3f'))
r2 = r2_score(y_test, y_predict)

print('RMSE =',RMSE,'\nR2 =', r2)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 24 is smaller than n_iter=100. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV 1/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=(train=-2427.747, test=-4152.947) total time=   0.1s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=(train=-2437.219, test=-5382.177) total time=   0.3s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=(train=-2424.299, test=-4498.615) total time=   0.4s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=(train=-2493.081, test=-4770.751) total time=   0.1s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=100;, score=(train=-2522.829, test=-3947.298) total time=   0.1s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=500;, score=(train=-1043.777, test=-3977.200) total time=   0.4s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=3, n_estimators=500;, score=(train=-1264.330, test=-5014.282) tota

In [23]:
#Lakukan optimasi hiperparameter menggunakan Bayesian optimization, pilih nilai yang masuk akal untuk max_depth,
#learning_rate, n_estimators. Gunakan 5-folds cross validation dan 100 iterasi.
#xgb dengan bayesian
! pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.4 MB/s eta 0:00:00


In [24]:
from skopt import BayesSearchCV

search_space = {
    'learning_rate' : (0.01, 1.0, 'log-uniform'),
    'max_depth' : (4, 20, 'log-uniform'),
    'n_estimators' : (2, 100, 'log-uniform')}

In [25]:
import xgboost as xgb
model = xgb.XGBRegressor(objective= 'reg:squarederror')

xgb_bayes_search = BayesSearchCV(model,
                                 search_space,
                                 n_iter = 100,
                                 scoring = 'neg_mean_absolute_error',
                                 cv = 5)

In [26]:
xgb_bayes_search.fit(X_train, y_train)

y_predict = xgb_bayes_search.predict(X_test)

In [27]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt

RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict)), '.3f'))
r2 = r2_score(y_test, y_predict)

print('RMSE =', RMSE, '\nR2 =', r2)

RMSE = 5135.562 
R2 = 0.8955800834340997


In [28]:
#Bandingkan 3 strategi pengoptimalan menggunakan RMSE dan R2

**TANPA OPTIMASI**

RMSE=  6128.629

R2=  0.8512921025017814 = **85%**

**GRID SEARCH CV**

RMSE = 5366.941

R2 = 0.8859590305351258 = **88%**

**RANDOM SEARCH CV**

RMSE = 5366.941

R2 = 0.8859590305351258 = **88%**

**BAYES SEARCH CV**

RMSE = 5135.562

R2 = 0.8955800834340997 = **89%**

**Maka untuk data ini metode terbaik yang menghasilkan nilai R2 terbaik ialah Bayes Search CV**